In [1]:
import os
import shutil
from pathlib import Path

# Define source and destination directories
source_dir = Path('CAESAR/output_dataset')
destination_dir = Path('data/segmentation_dataset')

# Create subdirectories in the destination
for i in range(10):
    # Each subfolder gets an 'images' and 'masks' subdirectory
    os.makedirs(destination_dir / f'subfolder_{i+1}' / 'images', exist_ok=True)
    os.makedirs(destination_dir / f'subfolder_{i+1}' / 'masks', exist_ok=True)

# Function to distribute matching images and masks
def distribute_files():
    # Get all image files in the source images subdirectory
    image_files = os.listdir(source_dir / 'images')
    num_files = len(image_files)
    
    # Iterate through image files
    for index, image_name in enumerate(image_files):
        mask_name = image_name  # Assuming mask names match image names exactly

        # Determine which subfolder to use based on file index
        subfolder_index = index % 10  # This will distribute files evenly across 10 subfolders

        # Define the target subdirectories for images and masks
        target_image_subdir = destination_dir / f'subfolder_{subfolder_index+1}' / 'images'
        target_mask_subdir = destination_dir / f'subfolder_{subfolder_index+1}' / 'masks'

        # Move image
        shutil.move(str(source_dir / 'images' / image_name), str(target_image_subdir / image_name))
        # Move mask
        shutil.move(str(source_dir / 'masks' / mask_name), str(target_mask_subdir / mask_name))

distribute_files()

print("Files have been successfully moved and distributed, ensuring images and masks match in each subfolder.")

Files have been successfully moved and distributed, ensuring images and masks match in each subfolder.


In [6]:
import os
import shutil
from math import ceil

# Define source and target directories
segmentation_src_dir = 'CAESAR/processed_dataset/setA/segmentation'
scans_src_dir = 'CAESAR/processed_dataset/setA/scans'
base_dst_dir = 'data/3D_reconstruction'

# Create 10 subfolders under data/3D_reconstruction, each with an images and scans folder
num_subfolders = 10
for i in range(1, num_subfolders + 1):
    os.makedirs(os.path.join(base_dst_dir, f'subfolder_{i}', 'images'), exist_ok=True)
    os.makedirs(os.path.join(base_dst_dir, f'subfolder_{i}', 'scans'), exist_ok=True)

# Function to distribute files among the subfolders ensuring corresponding scans and images are together
def distribute_files():
    scan_files = [f for f in os.listdir(scans_src_dir) if f.endswith('.ply')]
    num_files = len(scan_files)
    files_per_folder = ceil(num_files / num_subfolders)

    for i, scan_filename in enumerate(scan_files):
        base_name = os.path.splitext(scan_filename)[0]  # Get the base name without extension
        front_view_filename = f"{base_name}_front_view.png"
        side_view_filename = f"{base_name}_side_view.png"

        # Check if corresponding images exist
        if not (os.path.exists(os.path.join(segmentation_src_dir, front_view_filename)) and
                os.path.exists(os.path.join(segmentation_src_dir, side_view_filename))):
            print(f"Corresponding images for {scan_filename} not found. Skipping.")
            continue

        # Determine the destination subfolder index
        subfolder_index = (i // files_per_folder) + 1

        # Define source and destination paths
        scan_src_path = os.path.join(scans_src_dir, scan_filename)
        front_view_src_path = os.path.join(segmentation_src_dir, front_view_filename)
        side_view_src_path = os.path.join(segmentation_src_dir, side_view_filename)

        scan_dst_path = os.path.join(base_dst_dir, f'subfolder_{subfolder_index}', 'scans', scan_filename)
        front_view_dst_path = os.path.join(base_dst_dir, f'subfolder_{subfolder_index}', 'images', front_view_filename)
        side_view_dst_path = os.path.join(base_dst_dir, f'subfolder_{subfolder_index}', 'images', side_view_filename)

        # Copy files to destination
        shutil.copy(scan_src_path, scan_dst_path)
        shutil.copy(front_view_src_path, front_view_dst_path)
        shutil.copy(side_view_src_path, side_view_dst_path)

# Distribute the files
distribute_files()

print("Files have been copied and distributed successfully.")

Files have been copied and distributed successfully.
